In [1]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

In [2]:
import pandas as pd
import numpy as np

In [3]:
Xdata  = pd.read_csv('Xdata.csv')#publication data
Ydata  = pd.read_csv('Ydata.csv')#EV_sales data
Ydata['Years'] = Ydata['Years'].astype('int64')
idx_of_invalid_years = Xdata[Xdata['Year'].isnull()].index
Xdata.drop(idx_of_invalid_years,inplace=True)
Xdata['Year'] = Xdata['Year'].astype('int64')

In [4]:
#Xdata.head(20)
Ydata.head(30)

,Years,Total Citations per Year,Keyword_SEI,Keyword_cathode,Keyword_anode,Keyword_Efficiency,Keyword_electrode,Ci_5IPub,EV_Sales
0,1997,3819.0,787.0,183.0,635.0,0.0,1230.0,3564.0,0
1,1998,11862.0,1641.0,870.0,1832.0,0.0,2161.0,11036.0,0
2,1999,12632.0,3726.0,469.0,2120.0,49.0,3697.0,11735.0,17
3,2000,18320.0,1612.0,309.0,11235.0,0.0,10986.0,17070.0,9350
4,2001,25058.0,7802.0,5536.0,6234.0,176.0,5795.0,22078.0,20282
5,2002,23792.0,6552.0,3572.0,4739.0,7.0,7108.0,21639.0,36042
6,2003,24569.0,10066.0,2250.0,6302.0,323.0,4616.0,23795.0,47566
7,2004,27065.0,11537.0,4260.0,7379.0,601.0,8598.0,25157.0,84233
8,2005,31693.0,8870.0,3508.0,5533.0,0.0,5693.0,29734.0,205876
9,2006,42911.0,13445.0,5101.0,8163.0,247.0,13573.0,36222.0,251864


In [5]:
Xdata['Feed'] =  Xdata['Title'] + '. '+ Xdata['Abstract']
# for i in range(len(Xdata)):    
#       Xdata.iloc[i,17] = Xdata.iloc[i,17].ljust(Xdata['Feed'].str.len().max(),'a')#Padding Title
# Word_cloud = []
# for i in Xdata['Feed']:
#      for j in i:
#          Word_cloud.append(ord(j))
substr = {'1997': ' ','1998': ' ','1999': ' ','2000': ' ','2001': ' ','2002': ' ','2003': ' ','2004': ' ','2005': ' ',\
          '2006': ' ','2007': ' ','2008': ' ', '2009': ' ','2010': ' ','2011': ' ','2012': ' ','2013': ' ','2014': ' ',\
         '2015': ' ','2016': ' ','2017': ' ','2018': ' ','2019': ' '}
for i in range(Xdata['Year'].min(),Xdata['Year'].max()+1):
    for j in Xdata[Xdata['Year'] == i]['Feed']:
        try:
            substr[str(i)] += '. '+ j
        except:
            pass
        
###########    padding the string in each column to make them same length
max_l = len(substr['1997'])
for key in substr.keys():
    if max_l < len(substr[key]):
        max_l = len(substr[key])   
for key in substr.keys():
    substr[key] = substr[key].ljust(max_l,'a')    
            

In [6]:
subnum_copy = {'1997': [],'1998': [],'1999': [],'2000': [],'2001': [],'2002': [],'2003': [],'2004': [],'2005': [],\
          '2006': [],'2007': [],'2008': [], '2009': [],'2010': [],'2011': [],'2012': [],'2013': [],'2014': [],\
         '2015': [],'2016': [],'2017': [],'2018': [],'2019': []}
for key in substr.keys():
    for string in substr[key]:
        subnum_copy[key].append(ord(string))

In [7]:
len(subnum_copy['2002'])

103882

In [8]:
#collecting all x and y
all_x = []
all_y = []
for key in substr.keys(): 
    try:
        subnum_copy[key].append(Ydata[Ydata['Years']==int(key)]['Total Citations per Year'].values[0])
    except:
        pass
    temp_x = subnum_copy[key]
    temp_y = Ydata[Ydata['Years']==int(key)]['EV_Sales'].values[0]
    all_x.append(temp_x)
    #train_y2 = np.append(train_y2, np.asarray(train_y).astype('float64').reshape(-1, 1))
    all_y.append(temp_y)
print(len(all_x))


23


In [9]:
all_y[-1]

727390

## Train the x_input vs. y_input up to year of 2018

In [10]:
import numpy as np

# def vectorize_sequences(sequences, dimension=10000):
#     # Create an all-zero matrix of shape (len(sequences), dimension)
#     results = np.zeros((len(sequences), dimension))
#     for i, sequence in enumerate(sequences):
#         results[i, sequence] = 1.  # set specific indices of results[i] to 1s
#     return results

# # Our vectorized training data
# x_train = vectorize_sequences(train_data)
# # Our vectorized test data
# x_test = vectorize_sequences(test_data)
#train the x input vs. y input 
Xtrainwhole = np.asarray(all_x[:-1]).astype('float64')#.reshape(23, 1)
Ytrainwhole = np.asarray(all_y[:-1]).astype('float64').reshape(-1, 1)

In [18]:
len(substr['2015'])

103882

## Start of Training, validation and testing

In [29]:
scaler1 = MinMaxScaler()#this is a must
train_x = scaler1.fit_transform(Xtrainwhole)
scaler2 = MinMaxScaler()
train_y = scaler2.fit_transform(Ytrainwhole)
def build_model(neurons=32):
    # Because we will need to instantiate
    # the same model multiple times,
    # we use a function to construct it.
    
    model = models.Sequential() 
    
    model.add(layers.Dense(256, activation='relu',
                           input_shape=(train_x.shape[1],)))
    model.add(layers.Dense(16, activation='relu'))
    #model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(6, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

## k-fold cross validation of the model performance

In [ ]:
import numpy as np

###number of folds
k = 4
num_val_samples = len(train_x) // k
num_epochs = 50
all_scores = []
for i in range(k):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    val_data = train_x[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_y[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_train_x = np.concatenate(
        [train_x[:i * num_val_samples],
         train_x[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_y = np.concatenate(
        [train_y[:i * num_val_samples],
         train_y[(i + 1) * num_val_samples:]],
        axis=0)

    # Build the Keras model (already compiled)
    model = build_model()
    # Train the model (in silent mode, verbose=0)
    model.fit(partial_train_x, partial_train_y,
              epochs=num_epochs, batch_size=1, verbose=0)
    # Evaluate the model on the validation data
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

In [30]:
num_epochs = 100

model_v1 = build_model()
    # Train the model (in silent mode, verbose=0)
model_v1.fit(train_x, train_y,
              epochs=num_epochs, batch_size=1, verbose=0)

    # Evaluate the model on the test  data
predictions_2019 = model_v1.predict(np.asarray([all_x[-1]]).astype('float64'))
print(scaler2.inverse_transform(predictions_2019))

[[274701.72]]


In [32]:
predictions_2019

283613.3907406

In [28]:
from tensorflow.keras import backend as K

# Some memory clean-up
K.clear_session()


## Optimize the model to some extent

In [ ]:
num_epochs = 100
all_mae_histories = []
k = 10
for i in range(k):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    val_data = train_x[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_y[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_train_x = np.concatenate(
        [train_x[:i * num_val_samples],
         train_x[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_y = np.concatenate(
        [train_y[:i * num_val_samples],
         train_y[(i + 1) * num_val_samples:]],
        axis=0)

    # Build the Keras model (already compiled)
    model = build_model()
    # Train the model (in silent mode, verbose=0)
    history = model.fit(partial_train_x, partial_train_y,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=1, verbose=0)
    mae_history = history.history['val_mae']
    all_mae_histories.append(mae_history)

In [ ]:
average_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

### Visualizing the modelling error(Mean absolute error)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Epochs') 
plt.ylabel('Validation MAE')
plt.show()

In [ ]:
def smooth_curve(points, factor=0.9):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

smooth_mae_history = smooth_curve(average_mae_history[10:])

plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [ ]:
num_epochs = 100

model_v3 = build_model()
    # Train the model (in silent mode, verbose=0)
model_v3.fit(train_x, train_y,
              epochs=num_epochs, batch_size=1, verbose=0)

    # Evaluate the model on the test  data
predictions_2019 = model_v1.predict(np.asarray([all_x[-1]]).astype('float64'))
add_neuron = 32
loop_indicator = 1
while predictions_2019 < all_y[-1]-5000:
    add_neuron *= 2
    num_epochs += 50
    model_v1 = build_model(add_neuron)
    print('Processing loop: ', str(loop_indicator))
    model_v1.fit(train_x, train_y,
              epochs=num_epochs, batch_size=1, verbose=0)

    # Evaluate the model on the test  data
    predictions_2019 = model_v1.predict(np.asarray([all_x[-1]]).astype('float64'))
    loop_indicator += 1

In [ ]:
model_v1.predict(np.asarray([all_x[-1]]).astype('float64'))

In [ ]:
model_v5.summary()